# Video Player and Recorder with SSD Object Detection

# Configuration
- configure `video_config.json`
 - to set ssd weight file, classes and confidence
 - like below

```json
{
    "ssd_weights_path":"weights_SSD300.hdf5",
    "classes":["Aeroplane", "Bicycle", "Bird", "Boat", "Bottle",
               "Bus", "Car", "Cat", "Chair", "Cow", "Diningtable",
               "Dog", "Horse","Motorbike", "Person", "Pottedplant",
               "Sheep", "Sofa", "Train", "Tvmonitor"],
    "confidence":0.6
}
```

# How to use Video Player
- Video

```
player = VideoPlayer(source_path, verbose)
player.play(start, step, last)
```

- WebCam

```
player = VideoPlayer(0)
player.play()
```

In [1]:
from video_utils import VideoPlayer

Using TensorFlow backend.


In [2]:
# webcam mode
file_path = 0
player = VideoPlayer(file_path, verbose=0)

width: 640, height: 480
Total frames: -1
fps: 30


In [3]:
player.play()

Breaked.
Finished.


# How to use Video Recorder
- Video

```
recorder = VideoRecorder(source_path, verbose)
recorder.record(target_path, start, step, last)
```

- WebCam

```
recorder = VideoRecorder(0, verbose)
recorder.record(target_path)
```

In [4]:
from video_utils import VideoRecorder

In [5]:
# webcam mode
file_path = 0
recorder = VideoRecorder(file_path, verbose=0)

width: 640, height: 480
Total frames: -1
fps: 30


In [6]:
recorder.record('out.avi')

Breaked.
Finished.
